In [1]:
import dipy.data as dpd
import dipy.tracking.utils as dtu

In [2]:
dpd.fetch_bundles_2_subjects()

Data size is approximately 234MB
Dataset is already in place. If you want to fetch it again please first remove the folder /home/arokem/.dipy/exp_bundles_and_maps 


({'bundles_2_subjects.tar.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/38477/bundles_2_subjects.tar.gz',
   '97756fbef11ce2df31f1bedf1fc7aac7')},
 '/home/arokem/.dipy/exp_bundles_and_maps')

In [3]:
bd = dpd.read_bundles_2_subjects()

In [4]:
ls /home/ariel/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/

ls: cannot access '/home/ariel/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/': No such file or directory


In [5]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
DATA_SUB1 = '/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1'

In [7]:
import os.path as op
from glob import glob

In [8]:
bundle_fnames = glob(op.join(DATA_SUB1, 'bundles/*.trk'))

In [9]:
import nibabel as nib
t1_warped_img = nib.load(op.join(DATA_SUB1, 't1_warped.nii.gz'))

In [10]:
sl_sum = 0
for b_idx, bundle in enumerate(bundle_fnames):
    tgram = nib.streamlines.load(op.join(DATA_SUB1, 'bundles', bundle))
    print(bundle, len(tgram.streamlines))
    sl_sum += len(tgram.streamlines)


/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf1.right.trk 291
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_ifof.right.trk 1148
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf_3.right.trk 182
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_2.trk 953
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.right.trk 161
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_af.right.trk 2093
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.left.trk 343
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_1.trk 478
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cst.right.trk 2140
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_6.trk 3620
/home/a

In [11]:
((sl_sum, ) + t1_warped_img.shape + (1,))

(46156, 256, 256, 150, 1)

In [ ]:
bundle_arr = np.memmap('/home/arokem/bundlenet/data/bundles/npy/bundles.npy', mode='w+', 
                       shape=((sl_sum, ) + t1_warped_img.shape + (1,)), dtype=bool)
one_hot_arr = np.memmap('/home/arokem/bundlenet/data/bundles/npy/one_hot.npy', mode='w+', 
                        shape=(sl_sum, len(bundle_fnames)), dtype=bool)

In [ ]:
# fig = plt.figure()
# ax = fig.gca(projection='3d')
ii = 0
for b_idx, bundle in enumerate(bundle_fnames):
    tgram = nib.streamlines.load(op.join(DATA_SUB1, 'bundles', bundle))
    print(bundle, len(tgram.streamlines))
    sl_sum += len(tgram.streamlines)
    for sl_idx, sl in enumerate(list(dtu.move_streamlines(tgram.streamlines, np.linalg.inv(tgram.affine)))):
            if not np.mod(sl_idx, 100):
                print("Streamline {0} at index {1}".format(sl_idx, ii))
#         sl_npy_fname = op.join('/data/bundles/npy/','_'.join(op.split(bundle)[-1].split('.')[:2]) + '{0:05d}.npy'.format(sl_idx + 1))
#         if not op.exists(sl_npy_fname):
            vol = np.zeros(t1_warped_img.shape + (1,), dtype=bool)
            sl = np.round(sl).astype(int).T
            vol[sl[0], sl[1], sl[2]] = 1        
            bundle_arr[ii] = vol
            one_hot = np.zeros(len(bundle_fnames), dtype=bool)
            one_hot[b_idx] = 1
            one_hot_arr[ii] = one_hot
            ii += 1
#             np.save(sl_npy_fname, vol)

/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf1.right.trk 291
Streamline 0 at index 0
Streamline 100 at index 100
Streamline 200 at index 200
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_ifof.right.trk 1148
Streamline 0 at index 291
Streamline 100 at index 391
Streamline 200 at index 491
Streamline 300 at index 591
Streamline 400 at index 691
Streamline 500 at index 791
Streamline 600 at index 891
Streamline 700 at index 991
Streamline 800 at index 1091
Streamline 900 at index 1191
Streamline 1000 at index 1291
Streamline 1100 at index 1391
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf_3.right.trk 182
Streamline 0 at index 1439
Streamline 100 at index 1539
/home/arokem/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_2.trk 953
Streamline 0 at index 1621
Streamline 100 at index 1721
Streamline 200 at index 1821
Streamline 300 at index 1921
Streamline